In [95]:
fifa_year_as_strings = [str(int_) for int_ in fifa_range]
import re
for year in fifa_year_as_strings:
    if (re.search('[7-9]', year) != None):
        year = '0' + year
fifa_year_as_strings

['7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22']

In [31]:
from bs4 import BeautifulSoup
import requests

headers = {'User-Agent':
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}

fifa_year = input("What fifa year do you want to scrape? ")
fifa_range = range(int(fifa_year),23)
# Generate the results page URLs. this is for the top 300 potential rated players
results_pages = [f'https://sofifa.com/players?type=all&aeh=23&col=pt&sort=desc&r={fifa_year}0001&set=true&offset={i}' for i in range(0, 241, 60)]

# Create empty list to hold player page URLs
player_page_urls = []

# For each results page, scrape the URLs for each player page
for url in results_pages:
    response = requests.get(url, headers=headers)
    text = BeautifulSoup(response.text, 'html.parser')
    
    if response.status_code != 200:
        raise Exception(f'The status code is not 200! It is {response.status_code}.')

    # Get the book page URLs and add the domain, then add to player page URL list
    player_trs = text.find_all('tr')
    player_page_partial_urls = [tag.find('a').get('href') for tag in player_trs]
    complete_player_urls = [f'https://sofifa.com{end_part}' for end_part in player_page_partial_urls]
    complete_player_urls = complete_player_urls[1:] #first URL isn't a player
    
    # What is the difference between append and extend?
    player_page_urls.extend(complete_player_urls)

player_page_urls[30]

What fifa year do you want to scrape? 07


'https://sofifa.com/player/106850/mirko-vucinic/070001/'

In [5]:
player_page_urls[0:10]

['https://sofifa.com/player/158023/lionel-messi/070001/',
 'https://sofifa.com/player/136144/robinho/070001/',
 'https://sofifa.com/player/164859/theo-walcott/070001/',
 'https://sofifa.com/player/158232/salomon-kalou/070001/',
 'https://sofifa.com/player/153079/sergio-aguero/070001/',
 'https://sofifa.com/player/140957/oleg-husyev/070001/',
 'https://sofifa.com/player/54050/wayne-rooney/070001/',
 'https://sofifa.com/player/156616/franck-ribery/070001/',
 'https://sofifa.com/player/155862/sergio-ramos/070001/',
 'https://sofifa.com/player/153296/yoann-gourcuff/070001/']

In [34]:
import re
import get_stats

# Create an empty list to hold our player information
fifa_top_potential_player_list = []

for url in player_page_urls[0:2]:
    
     # We'll store our book information in dicitonaries so
    # we should create a new dict for every iteration
    
    player_dict = {}
    
    response = requests.get(url, headers = headers)
    text = BeautifulSoup(response.text, 'html.parser')
    
    if response.status_code != 200:
        raise Exception(f'The status code is not 200! It is {response.status_code}.')
        
    #Finding unique player ID for the URL
    li_tag = text.find('ul', attrs = {'class':'pl'}).find_all('li')
    player_id = list(li_tag[-1].strings)[1]
    
    #get player name
    
    name = re.search(f'/{player_id}/(.*)/{fifa_year}0001/', url)
    name = name.group(1)
    player_name = name.replace('-', ' ')
    player_name = player_name.title()
    
    #get known as player name
    known_as_player_name = text.find('header').find('h1').string
    
    #find the overall rating in fifa 16
    overall_rating = text.find('section', attrs = {'class':'card spacing'}).find_all('span')[0].string
    
    #Fifa 16 potential rating
    potential_rating = get_stats.find_potential_rating(text,overall_rating)
    
    #get position
    position = text.find('div', attrs = {'class':'meta ellipsis'}).find_all('span')
    positions = []
    for pos in position:
        positions.append(pos.string)
    
    #get nationality
    nationality = text.find('div', attrs = {'class':'meta ellipsis'})
    nationality = nationality.find('a').get('title')

    #get age
    age_text = text.find('div', attrs = {'class':'meta ellipsis'})
    age_text = list(age_text.strings)

    for x in age_text:
        if re.search('(..)y.o.', x) == None:
            continue
        else:
            age = re.search('(..)y.o.', x).group(1)
            break
    
    #get team
    result = text.find_all("div", class_="card")
    card = result[3].text.split('\n')
    for element in card:
        if element != '':
            team = element.strip()
            break
            
    # Store info in the dictionary
    player_dict['Player name'] = player_name
    player_dict['Known as'] = known_as_player_name
    player_dict['player_id'] = player_id
    player_dict['position'] = positions
    player_dict['nationality'] = nationality
    player_dict['team'] = team
    player_dict[f'Fifa {fifa_year} Potential rating'] = potential_rating

    #get and store fifa 17-22 overall ratings and ages
            
    #Get all urls for the specific player
    all_player_urls = [f'https://sofifa.com/player/{player_id}/{i}0001/' for i in fifa_range]
    #iterate through urls to get the overall ratings from each game
    for urls, i in zip(all_player_urls,fifa_range):
        if i == 7 or i == 8 or i ==9:
            i ='0'+str(i)
        response = requests.get(urls, headers = headers)
        playertext = BeautifulSoup(response.text, 'html.parser')
        other_overall_rating = playertext.find('section', attrs = {'class':'card spacing'}).find_all('span')[0].string
        if re.search('FIFA (..)', playertext.find('title').string).group(1) == str(i):
            player_dict[f'Fifa {i} overall_rating'] = other_overall_rating 
        else:
            player_dict[f'Fifa {i} overall_rating'] = None
            
        age_text = playertext.find('div', attrs = {'class':'meta ellipsis'})
        age_text = list(age_text.strings)
    
        for x in age_text:
            if player_dict[f'Fifa {i} overall_rating'] == None:
                player_dict[f'Fifa {i} age'] = None
                break
            elif re.search('(..)y.o.', x) == None:
                continue
            else:
                age = re.search('(..)y.o.', x).group(1)
                player_dict[f'Fifa {i} age'] = age
                break
    #append the player dictionary into the list
    fifa_top_potential_player_list.append(player_dict)


In [35]:
fifa_top_potential_player_list

[{'Player name': 'Lionel Messi',
  'Known as': 'L. Messi',
  'player_id': '158023',
  'position': ['RF'],
  'nationality': 'Argentina',
  'team': 'FC Barcelona',
  'Fifa 07 Potential rating': '94',
  'Fifa 07 overall_rating': None,
  'Fifa 07 age': None,
  'Fifa 08 overall_rating': None,
  'Fifa 08 age': None,
  'Fifa 09 overall_rating': None,
  'Fifa 09 age': None,
  'Fifa 10 overall_rating': '90',
  'Fifa 10 age': '22',
  'Fifa 11 overall_rating': '90',
  'Fifa 11 age': '23',
  'Fifa 12 overall_rating': '94',
  'Fifa 12 age': '24',
  'Fifa 13 overall_rating': '94',
  'Fifa 13 age': '25',
  'Fifa 14 overall_rating': '94',
  'Fifa 14 age': '26',
  'Fifa 15 overall_rating': '93',
  'Fifa 15 age': '27',
  'Fifa 16 overall_rating': '94',
  'Fifa 16 age': '28',
  'Fifa 17 overall_rating': '93',
  'Fifa 17 age': '29',
  'Fifa 18 overall_rating': '93',
  'Fifa 18 age': '30',
  'Fifa 19 overall_rating': '94',
  'Fifa 19 age': '31',
  'Fifa 20 overall_rating': '94',
  'Fifa 20 age': '32',
  'F

In [ ]:
import csv

fifaheaders = list(fifa_top_potential_player_list[0].keys())

# Create a new CSV file
with open(f'fifa_{fifa_year_as_string}_top300_potential_players.csv', 'a', encoding = 'utf-8', newline='') as csvfile:
    # Create the CSV writer object
    player_writer = csv.writer(csvfile)  
    # Write the initial header row
    player_writer.writerow(fifaheaders)
                           
    # Iterate over the list of dicts and write the values to CSV
    for player_dict in fifa_top_potential_player_list:
        player_writer.writerow(player_dict.values())